In [17]:
import string
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

import warnings
warnings.filterwarnings('ignore')

In [18]:
def normalize_address(address):
    # Define common words or city names to normalize
    normalization_dict = {
        'việt nam': '',
        'Việt Nam': ''
    }

    # Convert to lowercase for consistency
    address = address.lower()

    # Remove postal codes (e.g., 700000)
    address = re.sub(r'\d{6}', '', address)

    # Remove country names or specific place names if needed
    for key, value in normalization_dict.items():
        address = address.replace(key, value)

    # Tokenize the address by splitting on commas and removing extra spaces
    address_parts = [part.strip() for part in address.split(',')]

    # Normalize specific words (Phường, Quận, etc.)
    address_parts = [re.sub(r'\b(phường|quận|thành phố)\b', '', part) for part in address_parts]

    # Re-join the address parts back into a string
    normalized_address = ' '.join(address_parts)

    # Remove unnecessary punctuation
    normalized_address = re.sub(r'[^\w\s]', '', normalized_address)

    return normalized_address

def extract_date_desc(open_time_rule):
    try:
        # Convert string to JSON
        open_time_rule_json = json.loads(open_time_rule)
        # Extract dateDesc if the structure is correct
        if isinstance(open_time_rule_json, list) and len(open_time_rule_json) > 0:
            return open_time_rule_json[0].get('dateDesc', None)
    except (json.JSONDecodeError, TypeError):
        pass
    return "none"


def extract_rank_desc(rank_info):
    try:
        # Convert string to JSON
        rank_info_json = json.loads(rank_info)
        # Extract dateDesc if the structure is correct
        return rank_info_json.get('description', None)
    except (json.JSONDecodeError, TypeError):
        pass
    return "none"


extract_rank_desc('{"description": "Top 10% in the city"}')

'Top 10% in the city'

In [19]:
attraction_data = pd.read_csv('data/attractions_rows.csv')
attraction_type_data = pd.read_csv('data/attraction_types_rows.csv')
attraction_types_grouped = attraction_type_data.groupby('attraction_id')['type_name'].apply(list).reset_index()
# Merge data to associate attractions with their types
attractions = pd.merge(attraction_data, attraction_types_grouped, left_on="id", right_on="attraction_id")
attractions.drop(columns=['attraction_id','created_at', 'ename', 'video','location_id', 'phone'], inplace=True)

attractions['date_desc'] = attractions['open_time_rule'].apply(extract_date_desc)
attractions['rank_desc'] = attractions['rank_info'].apply(extract_rank_desc)
attractions.head(10)

,id,name,cover,images,hot_score,price,rank_info,latitude,longtitude,address,open_time_rule,description,avg_rating,rating_count,type_name,date_desc,rank_desc
0,78920,Quảng trường Ba Đình,https://ak-d.tripcdn.com/images/10011f000001h3...,"[""https://ak-d.tripcdn.com/images/10011f000001...",4.7,NaN,"{""jumpUrl"":""https://vn.trip.com/toplist/tripbe...",21.036750,105.835414,"Hùng Vương, Điện Biên, Ba Đình, Hà Nội 11812, ...","[{""dateDesc"":""Quanh năm"",""ruleType"":1,""dateRan...","Quảng trường Ba Đình, quảng trường lịch sử tại...",4.4,310,"[Quảng Trường, Di Tích Lịch Sử]",Quanh năm,Xếp hạng 13 trong danh sách Trải nghiệm nổi bậ...
1,78921,Hồ Hoàn Kiếm,https://ak-d.tripcdn.com/images/0105q12000etgn...,"[""https://ak-d.tripcdn.com/images/0105q12000et...",6.4,NaN,"{""jumpUrl"":""https://vn.trip.com/toplist/tripbe...",21.028667,105.852148,"Hàng Trống, Hoàn Kiếm, Hà Nội, Việt Nam","[{""dateDesc"":""Quanh năm"",""ruleType"":1,""dateRan...","Hồ Hoàn Kiếm, hồ nước nổi tiếng tại Hà Nội, nơ...",4.5,954,"[Hồ, Tham quan ban đêm]",Quanh năm,Xếp hạng 1 trong danh sách Trải nghiệm nổi bật...
2,78922,Văn Miếu – Quốc Tử Giám,https://ak-d.tripcdn.com/images/0106t120008ibn...,"[""https://ak-d.tripcdn.com/images/0106t120008i...",5.3,NaN,"{""jumpUrl"":""https://vn.trip.com/toplist/tripbe...",21.028118,105.835669,"58 P. Quốc Tử Giám, Văn Miếu, Đống Đa, Hà Nội,...","[{""dateDesc"":""Quanh năm"",""ruleType"":1,""dateRan...","Văn Miếu – Quốc Tử Giám, di tích lịch sử t...",4.5,291,"[Di Tích Lịch Sử, Đền Thờ/Miếu]",Quanh năm,Xếp hạng 8 trong danh sách Trải nghiệm nổi bật...
3,78923,Bảo tàng Mỹ thuật Việt Nam,https://ak-d.tripcdn.com/images/10070l000000dg...,"[""https://ak-d.tripcdn.com/images/10070l000000...",3.6,NaN,NaN,21.030784,105.837084,"66 P. Nguyễn Thái Học, Điện Biên, Ba Đình, Hà ...","[{""dateDesc"":""Quanh năm"",""ruleType"":1,""dateRan...",Bảo tàng Mỹ thuật Việt Nam là một bảo tàng tại...,4.4,110,"[Bảo Tàng, Bảo Tàng Nghệ Thuật]",Quanh năm,none
4,78926,Bảo tàng Chứng tích Chiến tranh,https://ak-d.tripcdn.com/images/100w0a0000004u...,"[""https://ak-d.tripcdn.com/images/100w0a000000...",5.6,NaN,"{""jumpUrl"":""https://vn.trip.com/toplist/tripbe...",10.779511,106.692092,"Phường 6, Quận 3, Thành phố Hồ Chí Minh 700000...","[{""dateDesc"":""Quanh năm"",""ruleType"":1,""dateRan...",Bảo tàng Chứng tích Chiến tranh tại TP.HCM là ...,4.4,338,"[Địa Điểm Quân Sự, Bảo Tàng]",Quanh năm,Xếp hạng 10 trong danh sách Trải nghiệm nổi bậ...
5,78927,Dinh Độc Lập,https://ak-d.tripcdn.com/images/0104y12000etgt...,"[""https://ak-d.tripcdn.com/images/0104y12000et...",6.3,NaN,"{""jumpUrl"":""https://vn.trip.com/toplist/tripbe...",10.776994,106.695302,"135 Đ. Nam Kỳ Khởi Nghĩa, Phường Bến Thành, Qu...","[{""dateDesc"":""Quanh năm"",""ruleType"":1,""dateRan...",Dinh Độc Lập tại TP.HCM là một công trình lịch...,4.3,465,[Kiến Trúc Lịch Sử],Quanh năm,Xếp hạng 4 trong danh sách Trải nghiệm nổi bật...
6,78928,Chợ Lớn,https://ak-d.tripcdn.com/images/100g0z000000nj...,"[""https://ak-d.tripcdn.com/images/100g0z000000...",3.2,NaN,NaN,10.754497,106.658341,"phường 11, Quận 5, Thành phố Hồ Chí Minh, Việt...","[{""dateDesc"":""Quanh năm"",""ruleType"":1,""dateRan...","Chợ Lớn là khu chợ nổi tiếng của Sài Gòn, nơi ...",4.1,78,[Khu Dân Cư Nổi Bật],Quanh năm,none
7,78967,Nhà thờ Đức Bà Sài Gòn,https://ak-d.tripcdn.com/images/100f0z000000ng...,"[""https://ak-d.tripcdn.com/images/100f0z000000...",5.7,NaN,NaN,10.779786,106.699019,"01 Công xã Paris, Bến Nghé, Quận 1, Thành phố ...",NaN,"Nhà thờ Đức Bà Sài Gòn, công trình kiến trúc G...",4.4,887,"[Nhà Thờ & Thánh Đường, Tham quan ban đêm]",none,none
8,78968,Lăng Chủ tịch Hồ Chí Minh,https://ak-d.tripcdn.com/images/100q0700000028...,"[""https://ak-d.tripcdn.com/images/100q07000000...",5.3,NaN,"{""jumpUrl"":""https://vn.trip.com/toplist/tripbe...",21.036897,105.834667,"Hùng Vương, Điện Biên, Ba Đình, Hà Nội, Việt Nam","[{""dateDesc"":""1 thg 11, 2024-31 thg 3, 2025"",""...","Lăng Chủ tịch Hồ Chí Minh, lăng mộ của Chủ tịc...",4.5,295,"

In [20]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assume `df` is your DataFrame with attraction data

# Parameters for weighted average calculation
overall_avg_rating = attractions['avg_rating'][attractions['avg_rating'] != 0].mean()
attractions.loc[(attractions['avg_rating'] == 0) & (attractions['rating_count'] == 0), 'avg_rating'] = overall_avg_rating

R = attractions['avg_rating']
v = attractions['rating_count']
m = attractions['rating_count'].quantile(1)  # Minimum threshold for popularity
C = attractions['avg_rating'].mean()           # Average rating across all attractions

# Calculate weighted average
attractions['weighted_average'] = (R * v + C * m) / (v + m)

# Normalize `hot_score` and `weighted_average`
scaler = MinMaxScaler()
scaled = scaler.fit_transform(attractions[['hot_score', 'weighted_average']])
# Create the weighted_df with 'id' column before setting the index
weighted_df = pd.DataFrame(scaled, columns=['hot_score', 'weighted_average'])
weighted_df['name'] = attractions['name']  # Add 'id' as a regular column
weighted_df.index = attractions['id']  # Use attraction names as the index

# Calculate the combined score
weighted_df['score'] = weighted_df['weighted_average'] * 0.4 + weighted_df['hot_score'] * 0.6

# Sort by score
weighted_df_sorted = weighted_df.sort_values(by='score', ascending=False)

# Check the result
weighted_df_sorted.head(10)



,hot_score,weighted_average,name,score
id,,,,
10521723,0.970588,1.000000,Bãi biển Nha Trang,0.982353
56816156,0.955882,0.731474,Cáp Treo Hòn Thơm,0.866119
23898595,1.000000,0.453401,Hòn Tre,0.781360
10523367,0.955882,0.511219,Bà Nà Hills,0.778017
36232888,0.955882,0.426369,hòn Thơm,0.744077
18832359,0.897059,0.447619,VinWonders Phú Quốc,0.717283
25487733,0.882353,0.440802,Vinpearl Safari Phú Quốc,0.705733
23029423,1.000000,0.203623,VinWonders Nha Trang,0.681449
137662207,0.852941,0.422098,Biển Thiên Cầm,0.680604


In [21]:
def separate(lisst):
    clean_text = []
    for t in lisst:
        cleaned = re.sub('\(.*\)', '', t) # Remove text inside parentheses
        cleaned = cleaned.translate(str.maketrans('','', string.digits))
        cleaned = cleaned.translate(str.maketrans('','', string.punctuation)).lower()
        clean_text.append(cleaned)
    return ' '.join(clean_text)



def remove_punc(text):
    cleaned = text.translate(str.maketrans('','', string.punctuation)).lower()
    clean_text = cleaned.translate(str.maketrans('','', string.digits))
    return clean_text

In [22]:
vietnamese_stopwords = []
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    vietnamese_stopwords = [line.strip() for line in file]

In [23]:
content_df = attractions[['id', 'address','type_name','date_desc','rank_desc','description','name']]
content_df['name'] = content_df['name'].apply(remove_punc)
content_df['description'] = content_df['description'].apply(remove_punc)
content_df['address'] = content_df['address'].apply(remove_punc)
content_df['type_name'] = content_df['type_name'].apply(separate)
content_df['rank_desc'] = content_df['rank_desc'].apply(remove_punc)
content_df['date_desc'] = content_df['date_desc'].apply(remove_punc)

content_df['bag_of_words'] = ''
content_df['bag_of_words'] = content_df[content_df.columns[1:]].apply(lambda x: ' '.join(x), axis=1)
content_df.set_index('id', inplace=True)

content_df = content_df[['bag_of_words']]

content_df.head()


,bag_of_words
id,
78920,hùng vương điện biên ba đình hà nội việt nam ...
78921,hàng trống hoàn kiếm hà nội việt nam hồ tham q...
78922,p quốc tử giám văn miếu đống đa hà nội việt n...
78923,p nguyễn thái học điện biên ba đình hà nội vi...
78926,phường quận thành phố hồ chí minh việt nam ...


In [24]:


new_df = pd.merge(weighted_df_sorted,content_df, left_index=True, right_index=True)


tfidf = TfidfVectorizer(stop_words=vietnamese_stopwords, min_df=5)
tfidf_matrix = tfidf.fit_transform(new_df['bag_of_words'].values.astype('str'))

tfidf_matrix.shape

new_df.head()

,hot_score,weighted_average,name,score,bag_of_words
id,,,,,
10521723,0.970588,1.000000,Bãi biển Nha Trang,0.982353,nha trang khánh hòa việt nam bãi biển quanh nă...
56816156,0.955882,0.731474,Cáp Treo Hòn Thơm,0.866119,gc bãi đất đỏ an thới phú quốc kiên giang việt...
23898595,1.000000,0.453401,Hòn Tre,0.781360,vĩnh nguyên nha trang khánh hòa việt nam đảobá...
10523367,0.955882,0.511219,Bà Nà Hills,0.778017,hoà ninh hòa vang đà nẵng việt nam núi tham qu...
36232888,0.955882,0.426369,hòn Thơm,0.744077,hòn thơm tp phú quốc kiên giang việt nam đảobá...


In [25]:
cos_sim = cosine_similarity(tfidf_matrix)
cos_sim.shape


(796, 796)

In [26]:
new_df.head()

,hot_score,weighted_average,name,score,bag_of_words
id,,,,,
10521723,0.970588,1.000000,Bãi biển Nha Trang,0.982353,nha trang khánh hòa việt nam bãi biển quanh nă...
56816156,0.955882,0.731474,Cáp Treo Hòn Thơm,0.866119,gc bãi đất đỏ an thới phú quốc kiên giang việt...
23898595,1.000000,0.453401,Hòn Tre,0.781360,vĩnh nguyên nha trang khánh hòa việt nam đảobá...
10523367,0.955882,0.511219,Bà Nà Hills,0.778017,hoà ninh hòa vang đà nẵng việt nam núi tham qu...
36232888,0.955882,0.426369,hòn Thơm,0.744077,hòn thơm tp phú quốc kiên giang việt nam đảobá...


In [27]:
def predict(title, similarity_weight=0.7, top_n=10):
    data = new_df.reset_index()
    index_att = data[data['name'] == title].index
    similarity = cos_sim[index_att].T
    
    sim_df = pd.DataFrame(similarity, columns=['similarity'])
    final_df = pd.concat([data, sim_df], axis=1)
    # You can also play around with the number
    final_df['final_score'] = final_df['score']*(1-similarity_weight) + final_df['similarity']*similarity_weight
    
    final_df_sorted = final_df.sort_values(by='final_score', ascending=False).head(top_n)
    final_df_sorted.set_index('name', inplace=True)
    return final_df_sorted[['score', 'similarity', 'final_score','bag_of_words']]

In [28]:
predict('Bãi biển Trà Cổ', similarity_weight=0.7, top_n=10)

,score,similarity,final_score,bag_of_words
name,,,,
Bãi biển Trà Cổ,0.379413,1.000000,0.813824,p trà cổ móng cái quảng ninh việt nam bãi biển...
Bãi biển Nha Trang,0.982353,0.394888,0.571127,nha trang khánh hòa việt nam bãi biển quanh nă...
Bãi biển sầm sơn,0.574722,0.545291,0.554120,pwwc sầm sơn thanh hoá việt nam bãi biển núi n...
Biển Thiên Cầm,0.680604,0.477045,0.538113,thôn chùa cẩm xuyên hà tĩnh việt nam bãi biển ...
Bãi biển Mỹ Khê,0.619607,0.480429,0.522182,bãi biển mỹ khê đà nẵng bãi biển quanh năm xếp...
Ky Co Beach,0.506349,0.522088,0.517367,nhơn lý thành phố qui nhơn bình định việt nam ...
Bãi biển Cửa Lò,0.636486,0.463406,0.515330,nghệ an việt nam bãi biển none none bãi biển c...
Bãi biển Ông Lang,0.427074,0.514176,0.488045,ong lang beach phu quoc island việt nam bãi b...
Bãi biển Non Nước,0.436129,0.486883,0.471657,hòa hải việt nam bãi biển quanh năm xếp hạng ...
